# Multinomial Naive Bayesian



In [38]:
# import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# load data, select features, train test split
data = pd.read_csv("hw4_naive.csv")
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# training multinomical naive bayes classifer. this function goes in and finds probability of being in a class and the probability of features showing up in a class. smoothing set to one by default
def train_naive(X, y, alpha=1.0):
  n_samples, n_features = X.shape
  # find unique classes
  classes = np.unique(y)
  n_classes = len(classes)
  class_priors = np.zeros(n_classes)
  # probability of being a class in the dataset
  for i, c in enumerate(classes):
    class_priors[i] = np.sum(y == c) / n_samples
   # find unique features
  feature_values = []
  for j in range(n_features):
    values = np.unique(X[:, j])
    feature_values.append(values)
  feature_probs = {}
  for i, c in enumerate(classes):
    X_c = X[y == c]
    count_c = X_c.shape[0]
    for j in range(n_features):
      for val in feature_values[j]:
        count = np.sum(X_c[:, j] == val)
        # with smoothing, P(feature|class)
        prob = (count + alpha) / (count_c + alpha * len(feature_values[j]))
        feature_probs[(i, j, val)] = prob
  return {'classes': classes,'class_priors': class_priors,'feature_probs': feature_probs}

In [40]:
# takes in model, uses it to run inference on input vector X
def predict_naive(model, X):
  # load in from model
  classes = model['classes']
  class_priors = model['class_priors']
  feature_probs = model['feature_probs']

  n_samples = X.shape[0]
  n_classes = len(classes)
  predictions = np.zeros(n_samples, dtype=int)
  # use log probability to not go into numerical underflow
  for i in range(n_samples):
    log_probs = np.zeros(n_classes)
    for j in range(n_classes):
      log_prob = np.log(class_priors[j])
      for k in range(X.shape[1]):
        feature_val = X[i, k]
        # feature probability in a given class
        if (j, k, feature_val) in feature_probs:
          prob = feature_probs[(j, k, feature_val)]
        else:
          # infinitesmial small
          prob = 1e-10
        log_prob += np.log(prob)
      log_probs[j] = log_prob
    # return class with highest probability
    predictions[i] = classes[np.argmax(log_probs)]
  return predictions

In [41]:
model = train_naive(X_train, y_train, alpha=1.0) #train on data
y_pred = predict_naive(model, X_test) #test on testing

In [42]:
accuracy = accuracy_score(y_test, y_pred) #calculate accuracy
f1 = f1_score(y_test, y_pred) #calculate f1

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8232
F1 Score: 0.7585


# Gaussian Naive Bayes

In [43]:
#train a gaussian naive bayes classifier
def gaussian_train(X, y):
  # find unique classes
  classes = np.unique(y)
  class_count = {}
  mean = {}
  var = {}
  for c in classes:
    # data points where we are in class c
    X_c = X[y == c]
    # number of data points
    class_count[c] = X_c.shape[0]
    # mean of features for a class
    mean[c] = np.mean(X_c, axis=0)
    # variance of features for a class
    var[c] = np.var(X_c, axis=0)
  return classes, class_count, mean, var

In [44]:
 #Calculate Gaussian probability density function
def calculate_likelihood(x, c, mean, var):
  mean_c = mean[c]
  var_c = var[c]
  #to prevent 0s
  epsilon = 1e-10
  var_c = var_c + epsilon
  likelihood = 1
  for i in range(len(x)):
    # pdf on var and mean
    exponent = np.exp(-((x[i] - mean_c[i])**2) / (2 * var_c[i]))
    likelihood *= (1 / np.sqrt(2 * np.pi * var_c[i])) * exponent
    #return likelihood of observing features x given class c
  return likelihood

In [45]:
# predict the class of a single sample x
def predict_one(x, classes, class_count, mean, var):
  posts = []
  for c in classes:
    # calculute probs for each class
    prior = class_count[c] / sum(class_count.values())
    likelihood = calculate_likelihood(x, c, mean, var)
    posterior = prior * likelihood
    posts.append(posterior)
  # return the highest prob
  return classes[np.argmax(posts)]

In [46]:
# inference, predicting the output of all samples
def gaussian_predict(X, classes, class_count, mean, var):
   predictions = []
   for x in X:
       predictions.append(predict_one(x, classes, class_count, mean, var))
   return np.array(predictions)

In [47]:
#training and inferencing
classes, class_count, mean, var = gaussian_train(X_train, y_train)
y_pred = gaussian_predict(X_test, classes, class_count, mean, var)

In [48]:
# accuracy and f1
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5973
F1 Score: 0.3135


# Comparison

Multinomial naive bayes seems to be significantly better than Gaussian naive bayes, likely due to the discrete nature of the data points

# K-means



In [49]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import defaultdict

In [50]:
# euclidian distance
def distance(point1, point2):
  return np.sqrt(np.sum((point1 - point2) ** 2))

In [51]:
#calculate centroid of a cluster of points, only uses mean
def calc_centroid(cluster_points, method='mean'):
  if len(cluster_points) == 0:
    return None
  # return the mean of each position
  return np.mean(cluster_points, axis=0)

In [52]:
# intitalize the centroids with 2 different methods
def initalize_centroids(data, k, initilization='random seed'):
  if initilization == 'random seed':
    # take random indeces
    index = random.sample(range(len(data)), k)
    return data[index]
  elif initilization == 'random split':
    # assign each one point to a random centroid
    cluster_assignments = np.random.randint(0, k, size=len(data))
    centroids = np.zeros((k, data.shape[1]))
    for i in range(k):
      cluster_points = data[cluster_assignments == i]
      centroids[i] = calc_centroid(cluster_points, 'mean')
    return centroids
  return None

In [53]:
def k_means_clustering(data, k, max_iter=100, method='mean', initilization='random seed'):
  #initalize clusters
  centroids = initalize_centroids(data, k, initilization)
  iterations = 0
  # last centroid for each point
  prev_assignments = None
  current_assignments = np.zeros(data.shape[0], dtype=int)
  # measures if previous assignments are diff as current
  changed = True
  # not converged and more iterations to go
  while iterations < max_iter and changed:
    distances = np.zeros(k)
    # for each data point
    for i in range(data.shape[0]):
      # for each centroid
      for j in range(k):
        # calculate distance from current point to each centroid
        distances[j] = distance(data[i], centroids[j])
      # assign to the closest centroid
      current_assignments[i] = np.argmin(distances)
    # converged
    if prev_assignments is not None and np.array_equal(prev_assignments, current_assignments):
      changed = False
    # change previous assignments as we prepare for another loop
    prev_assignments = current_assignments.copy()
    # calculate the new mean of the k centroids
    for j in range(k):
      cluster_points = data[current_assignments == j]
      centroids[j] = calc_centroid(cluster_points, method)
    # increase the iterations by 1 as we continues
    iterations += 1
  # assigning final clusters and the center position of each cluster
  clusters = []
  # for each center, add the points assigned there to the appropriate cluster
  for i in range(k):
    mask = current_assignments == i
    clusters.append(data[mask])
  # return the clusters of points and the centroids themselves
  return clusters, centroids

In [54]:
# run it on the given data
clusters = pd.read_csv('hw4_cluster.csv')
clusters.head()
clusters= clusters.values

In [55]:
#find clusters and centroids on data
final_clusters, centroids = k_means_clustering(clusters, k=5, max_iter=50, method='mean', initilization='random seed')

In [56]:
# function to calculate silohouette scors
def silhouette_score(clusters):
  # Combine all clusters into one big array and create mapping
  all_points = np.vstack(clusters) if len(clusters) > 0 else np.array([])
  n = all_points.shape[0]
  point_to_cluster = {}
  point_index = 0
  for cluster_idx, cluster in enumerate(clusters):
    for _ in range(len(cluster)):
      point_to_cluster[point_index] = cluster_idx
      point_index += 1

  silhouette_values = []
  for i in range(n):
    point = all_points[i]
    cluster_idx = point_to_cluster[i]
    # Calculate average distance to points in the same cluster
    a_i = 0
    same_cluster_points = 0
    for j in range(n):
      if i != j and point_to_cluster[j] == cluster_idx:
        a_i += distance(point, all_points[j])
        same_cluster_points += 1

    if same_cluster_points > 0:
      a_i /= same_cluster_points
    else:
      a_i = 0
    # Calculate minimum avg distance to points in diff clusters
    b_i = float('inf')

    for other_cluster_idx in range(len(clusters)):
      if other_cluster_idx != cluster_idx:
        avg_distance = 0
        other_cluster_points = 0
        for j in range(n):
          if point_to_cluster[j] == other_cluster_idx:
            avg_distance += distance(point, all_points[j])
            other_cluster_points += 1
        if other_cluster_points > 0:
          avg_distance /= other_cluster_points
          b_i = min(b_i, avg_distance)

    if b_i == float('inf'):
      b_i = 0

    if a_i == 0 and b_i == 0:
      silhouette_i = 0
    else:
      silhouette_i = (b_i - a_i) / max(a_i, b_i)

    silhouette_values.append(silhouette_i)
  # Return the average silhouette value across all points
  return np.mean(silhouette_values)

score = silhouette_score(final_clusters)
print(f"Silhouette score for k=5: {score:.4f}")

Silhouette score for k=5: 0.5901


# Bonus

In [59]:
highest = -2 #outside of silhouette range
k=-1
#loop through all ks asked for
for i in range(2, 6):
  final_clusters, centroids = k_means_clustering(clusters, k=i, max_iter=50, method='mean', initilization='random seed')
  score = silhouette_score(final_clusters)
  if score > highest:
    highest = score
    k = i
  print(f"Silhouette score for k={i}: {score:.4f}")

print(f"\n\n\n\nHighest Silhouette score is: {highest:.4f} when k={k}")

Silhouette score for k=2: 0.7689
Silhouette score for k=3: 0.7192
Silhouette score for k=4: 0.6294
Silhouette score for k=5: 0.5901




Highest Silhouette score is: 0.7689 when k=2


K=2 appears to be the best as it has the highest Silhouette score of any k in range from k=2 to k=5